In [1]:
import hyperSLIC
import hyperspy.api as hs
import numpy as np
import matplotlib.pyplot as plt
import time
import multiprocessing as mp
import numba as nb
from numba import jit
from tqdm import tqdm
import os
import copy
%matplotlib qt6

In [2]:
s = hs.load(r"C:\Users\tas72\Downloads\SimulatedData\SimulatedData\data\SEND.hspy",lazy=True)

In [3]:
edx = hs.load(r"C:\Users\tas72\Downloads\SimulatedData\SimulatedData\data\EDS.hspy")

In [4]:
s.data = s.data.astype('uint16')

In [5]:
s.compute()

[########################################] | 100% Completed | 90.27 s


In [6]:
def plot_ranges(raveled,percentile,bins, mode, plot_or_not):
    reshaped = np.reshape(raveled, (np.shape(raveled)[0]*np.shape(raveled)[1], np.shape(raveled)[2]))
    if mode == 'range':
        ranges = np.ptp(reshaped, axis=0)
    if mode == 'var':
        ranges = np.var(reshaped,axis=0)
    percentile = np.percentile(ranges,percentile)
    print(percentile)
    if plot_or_not == True:
        fig,ax = plt.subplots()
        ax.hist(ranges,density = True, bins = bins)
        plt.yscale('log')

        ax.axvline(percentile,linestyle = ":",color ='r' )

@jit(nopython = True)
def quick_wheat(ranges, reshaped, percentile):
    wheat = reshaped.T[ranges >= percentile]
    return wheat.T

def wheat_from_chaff(data, percentile,mode):
    reshaped = np.reshape(data, (np.shape(data)[0]*np.shape(data)[1], np.shape(data)[2]))
    print(np.shape(reshaped))
    print('Reshaped')
    if mode == 'range':
        ranges = np.ptp(reshaped, axis=0)
    if mode == 'var':
        ranges = np.var(reshaped,axis=0)
    ranges = np.ptp(reshaped, axis=0)
    print(np.shape(ranges))
    print('Ranges aquried')
    percentile = np.percentile(ranges,percentile)
    print(percentile)
    wheat = quick_wheat(ranges, reshaped, percentile)
    print(np.shape(wheat))
    print('Wheat separated')
    maxes = np.amax(wheat, axis=0)
    minis = np.amin(wheat,axis=0) 
    
    wheat = wheat - minis

    wheat = wheat/maxes
    
    wheat = np.reshape(wheat,(np.shape(data)[0],np.shape(data)[1],-1))
    return wheat

In [7]:
raveled = np.reshape(s.data, (s.axes_manager[1].size, s.axes_manager[0].size,s.axes_manager[2].size*s.axes_manager[3].size))

In [112]:
plot_ranges(raveled,98,50,'range',True)

2.0


In [8]:
wheat_edx = wheat_from_chaff(edx.data,95,'range')
wheat_edx = hs.signals.Signal1D(wheat_edx)

(6400, 1024)
Reshaped
(1024,)
Ranges aquried
5.0
(6400, 66)
Wheat separated


In [9]:
wheat = wheat_from_chaff(raveled,98,'range')
print('Complete', np.shape(wheat))

(6400, 1034289)
Reshaped
(1034289,)
Ranges aquried
2.0
(6400, 72968)
Wheat separated
Complete (80, 80, 72968)


In [10]:
sed_weight_factor = 0.2
edx_weight_factor = 3

In [11]:
wheat_hs = hs.signals.Signal1D(np.append(sed_weight_factor*wheat,edx_weight_factor*wheat_edx.data,axis=2))

In [14]:
epochs = 5 #(5 works okay with these params)
iterations = 5
cluster_number = 100
m_value = 1
searchspace = 1
method = 'semi'
test = hyperSLIC.SLIC(wheat_hs,method,cluster_number,m_value,searchspace)

In [15]:
accumulator = np.zeros((test.width,test.height,test.k))
for k in range(epochs):
    test = hyperSLIC.SLIC(wheat_hs,method,cluster_number,m_value,searchspace) # centroid initalisation, seed_num, m, searchspace
    for i in tqdm(range(iterations)):
        test.find_closest_centeroid()
        test.update_centeroids()
    for i in range(np.shape(accumulator)[0]):
        for j in range(np.shape(accumulator)[1]):
            cluster = test.closest_centeroid[i,j]
            accumulator[i,j,int(cluster)] += 1

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:34<00:00,  6.91s/it]


In [18]:
test.closest_centeroid = np.argmax(accumulator,axis=2)

In [16]:
vals = np.linspace(0,1,256)
np.random.shuffle(vals)
cmap = plt.cm.colors.ListedColormap(plt.cm.gist_stern(vals))
centeroids = np.zeros((test.width,test.height))
for counter,centeroid in enumerate(test.xy_centeroids):
    centeroids[int(centeroid[0]),int(centeroid[1])] = counter    
fig,ax = plt.subplots()
ax.imshow(centeroids,cmap=cmap)

In [19]:
vals = np.linspace(0,1,256)
np.random.shuffle(vals)
cmap = plt.cm.colors.ListedColormap(plt.cm.gist_stern(vals))
fig,ax = plt.subplots(1,3)
ax[0].imshow(test.image,cmap='gray')
ax[1].imshow(test.closest_centeroid,cmap=cmap)
ax[2].imshow(test.image,cmap='gray')
ax[2].imshow(test.closest_centeroid,cmap=cmap,alpha=0.25)
ax[0].set_xticks([])
ax[1].set_xticks([])
ax[2].set_xticks([])
ax[0].set_yticks([])
ax[1].set_yticks([])
ax[2].set_yticks([])
#plt.savefig('dark_feild_total',dpi=600)

[]

In [20]:
fig,ax = plt.subplots()
ax.imshow(np.argmax(accumulator,axis=2),cmap=cmap)

Check if regions are similar to eachother- and merge is this is so

In [21]:
summed_channels = np.zeros(test.k,dtype='object')
number_of_occurances = np.zeros(test.k)

for row in tqdm(range(np.shape(test.closest_centeroid)[0])):
    for col in range(np.shape(test.closest_centeroid)[1]):
        
        arg = int(test.closest_centeroid[row,col])
        number_of_occurances[arg] += 1 
        channel = test.dot_data[row,col].astype(np.float32)# Use the copy not the data with the masked bright feild
        summed_channels[arg]+=channel

for i in range(len(number_of_occurances)):
    occurances = number_of_occurances[i]
    if occurances == 0:
        number_of_occurances[i] = np.nan
        

mean_channels = summed_channels/number_of_occurances

100%|██████████████████████████████████████████████████████████████████████████████████| 80/80 [00:04<00:00, 18.45it/s]


In [22]:
combine_threshold = 0.975
from scipy.stats import pearsonr
updated_centroids = copy.deepcopy(test.closest_centeroid)
for i in tqdm(range(len(mean_channels))):
    channel_oi = mean_channels[i]
    for counter,comparison_channel in enumerate(mean_channels[i:]):
        difference = channel_oi - comparison_channel
        difference = np.abs(difference)
        difference = np.sum(difference)
        try:
            r = pearsonr(channel_oi,comparison_channel)
            if r[0] > combine_threshold:
                args = np.where(updated_centroids == i)
                for j in range(len(args[0])):
                        row = args[0][j]
                        col = args[1][j]
                        updated_centroids[row,col] = i+counter
        except:
            pass
                
fig,ax = plt.subplots(1,3)
ax[0].imshow(test.image,cmap='gray')
ax[1].imshow(test.closest_centeroid,cmap=cmap)
ax[2].imshow(test.image,cmap='gray')
ax[2].imshow(updated_centroids,cmap=cmap,alpha=0.5)


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:05<00:00, 16.71it/s]


In [121]:
summed_patterns = np.zeros(test.k,dtype='object')

number_of_occurances = np.zeros(test.k)


for row in tqdm(range(np.shape(test.closest_centeroid)[0])):
    for col in range(np.shape(test.closest_centeroid)[1]):
        
        arg = int(test.closest_centeroid[row,col])
        number_of_occurances[arg] += 1 
        pattern = s.data[row,col].astype(np.float32)# Use the copy not the data with the masked bright feild
        summed_patterns[arg]+=pattern

for i in range(len(number_of_occurances)):
    occurances = number_of_occurances[i]
    if occurances == 0:
        number_of_occurances[i] = -1
        

mean_patterns = summed_patterns/number_of_occurances

100%|██████████████████████████████████████████████████████████████████████████████████| 80/80 [00:11<00:00,  6.80it/s]


In [122]:
grid_x, grid_y = 10,10
fig,ax = plt.subplots(grid_x,grid_y)
for i in range(grid_x):
    for j in range(grid_y):
        try:
            ax[i,j].imshow(mean_patterns[i*grid_x+j],vmax=0.1,cmap='Greys')
            ax[i,j].set_xticks([])
            ax[i,j].set_yticks([])
            #ax[i,j].title.set_text(i*25+j)

        except:
            ax[i,j].set_xticks([])
            ax[i,j].set_yticks([])

plt.tight_layout()
plt.subplots_adjust(left=0, right=1, top=1, bottom=0,wspace=-0, hspace=0)
#plt.savefig('all_patterns.png', dpi=300)

In [41]:
for pattern in mean_patterns:
    try:
        fig,ax = plt.subplots()
        ax.imshow(pattern,vmax=0.1)
        plt.waitforbuttonpress()
        plt.close()
    except:
        plt.close()


In [67]:
np.save('diffrcation_patterns',mean_patterns)

In [68]:
np.save('clusters',test.closest_centeroid)

In [ ]:
np.load()